In [1]:
# AGMA futtatasa

In [2]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/SZTAKI/")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
os.chdir("/content/drive/My Drive/SZTAKI/AGMA-Pose-Estimator-and-Sequence-Selector")

In [6]:
import os
import glob
import math
import torch
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
import cv2
import numpy as np
from torchvision import transforms
import matplotlib.pyplot as plt

class InfantPoseDetector_AGMA:
    def __init__(self):
        self.device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        self.hrnet_cfg = self._get_hrnet_config()
        self._init_models()

    #def _get_hrnet_config(self):
    def _get_hrnet_config(self):
        """
        Visszaadja a HRNet modell konfigurációját.
        """
        cfg = {
            'MODEL': {
                'NAME': 'pose_hrnet',
                'INIT_WEIGHTS': True,
                'PRETRAINED': '',
                'NUM_JOINTS': 17,
                'TAG_PER_JOINT': True,
                'TARGET_TYPE': 'gaussian',
                'IMAGE_SIZE': [192, 256],
                'HEATMAP_SIZE': [48, 64],
                'SIGMA': 2,
                'EXTRA': {
                    'STAGE2': {
                        'NUM_MODULES': 1,
                        'NUM_BRANCHES': 2,
                        'NUM_BLOCKS': [4, 4],
                        'NUM_CHANNELS': [32, 64],
                        'BLOCK': 'BASIC',
                        'FUSE_METHOD': 'SUM'
                    },
                    'STAGE3': {
                        'NUM_MODULES': 4,
                        'NUM_BRANCHES': 3,
                        'NUM_BLOCKS': [4, 4, 4],
                        'NUM_CHANNELS': [32, 64, 128],
                        'BLOCK': 'BASIC',
                        'FUSE_METHOD': 'SUM'
                    },
                    'STAGE4': {
                        'NUM_MODULES': 3,
                        'NUM_BRANCHES': 4,
                        'NUM_BLOCKS': [4, 4, 4, 4],
                        'NUM_CHANNELS': [32, 64, 128, 256],
                        'BLOCK': 'BASIC',
                        'FUSE_METHOD': 'SUM'
                    },
                    'FINAL_CONV_KERNEL': 1,
                    'PRETRAINED_LAYERS': ['*']
                }
            },
            'TEST': {
                'MODEL_FILE': './models/Darkpose32_retrained.pth',
                'BATCH_SIZE_PER_GPU': 1,
                'FLIP_TEST': False,
                'POST_PROCESS': True,
                'SHIFT_HEATMAP': False,
                'USE_GT_BBOX': False,
                'IMAGE_THRE': 0.1,
                'NMS_THRE': 0.6,
                'SOFT_NMS': False,
                'OKS_THRE': 0.5,
                'IN_VIS_THRE': 0.0,
                'COCO_BBOX_FILE': '',
                'BBOX_THRE': 1.0
            },
            'GPUS': [0] if torch.cuda.is_available() else [],
            'CUDNN': {
                'BENCHMARK': True,
                'DETERMINISTIC': False,
                'ENABLED': True
            },
            'DATASET': {
                'ROOT': '',
                'DATASET': 'coco',
                'TRAIN_SET': 'train2017',
                'TEST_SET': 'val2017',
                'DATA_FORMAT': 'jpg',
                'HYBRID_JOINTS_TYPE': '',
                'SELECT_DATA': False,
                'FLIP': True,
                'SCALE_FACTOR': 0.25,
                'ROT_FACTOR': 30,
                'PROB_HALF_BODY': 0.0,
                'NUM_JOINTS_HALF_BODY': 8,
                'COLOR_RGB': True
            },
        }
        return cfg

    def _init_models(self):
        """Modellek betöltése és inicializálása"""
        # Személy detektáló modell
        self.detection_model = fasterrcnn_resnet50_fpn(pretrained=True).to(self.device).eval()

        # Pózbecslő modell
        self.pose_model = self._load_pose_model()

    def _load_pose_model(self):
        """Pozíció becslő modell betöltése"""
        try:
            import lib.models.pose_hrnet as net
        except ImportError:
            import models.pose_hrnet as net

        pose_model = net.get_pose_net(self.hrnet_cfg, is_train=False)
        model_file = self.hrnet_cfg['TEST']['MODEL_FILE']

        if not os.path.exists(model_file):
            raise FileNotFoundError(f"Modell fájl nem található: {model_file}")

        if torch.cuda.is_available():
            pose_model.load_state_dict(torch.load(model_file), strict=False)
        else:
            pose_model.load_state_dict(torch.load(model_file, map_location='cpu'), strict=False)

        return pose_model.to(self.device).eval()

    def process_image(self, image_path, output_dir=None, save_visualization=True):
        """Egyetlen kép feldolgozása"""
        try:
            image = self._load_image(image_path)
            box = self._detect_person(image)
            if box is None:
                return None

            center, scale = self._box_to_center_scale(box)
            keypoints = self._estimate_pose(image, center, scale)

            if save_visualization and output_dir:
                self._visualize_pose(image_path, image, keypoints, output_dir)

            return keypoints

        except Exception as e:
            print(f"Hiba a kép feldolgozása során: {str(e)}")
            return None

    def process_batch(self, input_dir, output_dir, coords_file="result_coords.txt"):
        """Teljes könyvtár képeinek feldolgozása"""
        os.makedirs(output_dir, exist_ok=True)
        image_files = self._find_image_files(input_dir)

        if not image_files:
            print("Nincsenek feldolgozható képek!")
            return

        with open(os.path.join(output_dir, coords_file), 'w') as f:
            for i, img_path in enumerate(image_files, 1):
                print(f"Feldolgozás ({i}/{len(image_files)}): {os.path.basename(img_path)}")
                keypoints = self.process_image(img_path, output_dir)

                if keypoints is not None:
                    self._save_coordinates(f, img_path, keypoints)

    # Segédmetódusok
    def _load_image(self, image_path):
        """Kép betöltése és előkészítése"""
        image = cv2.imread(image_path)
        if image is None:
            raise FileNotFoundError(f"Kép nem található: {image_path}")
        return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    def _detect_person(self, image):
        """Személy detektálása a képen"""
        tensor = F.to_tensor(image).unsqueeze(0).to(self.device)
        with torch.no_grad():
            detections = self.detection_model(tensor)[0]

        person_indices = (detections['labels'] == 1) & (detections['scores'] > 0.7)
        if not person_indices.any():
            return None

        best_idx = torch.argmax(detections['scores'][person_indices])
        return detections['boxes'][person_indices][best_idx].cpu().numpy()

    def _box_to_center_scale(self, box):
        """Bounding box átalakítása középponttá és skálává"""
        center = np.array([(box[0] + box[2]) / 2, (box[1] + box[3]) / 2], dtype=np.float32)
        width = box[2] - box[0]
        height = box[3] - box[1]

        aspect_ratio = self.hrnet_cfg['MODEL']['IMAGE_SIZE'][0] / self.hrnet_cfg['MODEL']['IMAGE_SIZE'][1]
        if width > aspect_ratio * height:
            height = width / aspect_ratio
        elif width < aspect_ratio * height:
            width = height * aspect_ratio

        scale = np.array([width / 200.0, height / 200.0], dtype=np.float32)
        return center, scale

    def _estimate_pose(self, image, center, scale):
        """Pózbecslés a megadott képen"""
        rotation = 0
        trans = self._get_affine_transform(center, scale, rotation, self.hrnet_cfg['MODEL']['IMAGE_SIZE'])

        input_hrnet = cv2.warpAffine(
            image, trans,
            (int(self.hrnet_cfg['MODEL']['IMAGE_SIZE'][0]), int(self.hrnet_cfg['MODEL']['IMAGE_SIZE'][1])),
            flags=cv2.INTER_LINEAR
        )

        # Normalizálás és tensorrá alakítás
        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        input_tensor = normalize(F.to_tensor(input_hrnet)).unsqueeze(0).to(self.device)

        with torch.no_grad():
            output = self.pose_model(input_tensor)

        preds, _ = self._get_final_preds(
            output.clone().cpu().numpy(),
            np.asarray([center]),
            np.asarray([scale]),
        )

        return preds[0]

    def _visualize_pose(self, image_path, image, keypoints, output_dir):
        """Pozíció vizualizálása és mentése"""
        filename = os.path.basename(image_path)
        name, ext = os.path.splitext(filename)
        output_path = os.path.join(output_dir, f"{name}_pose{ext}")

        # COCO skeleton definíció
        skeleton = [
            [15,13],[13,11],[16,14],[14,12],[11,12],[5,11],[6,12],
            [5,6],[5,7],[6,8],[7,9],[8,10],[1,2],[0,1],[0,2],
            [1,3],[2,4],[3,5],[4,6]
        ]

        img_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Vonalak rajzolása
        for j1, j2 in skeleton:
            pt1 = tuple(map(int, keypoints[j1]))
            pt2 = tuple(map(int, keypoints[j2]))
            if all(0 <= x < img_bgr.shape[1] and 0 <= y < img_bgr.shape[0] for x, y in [pt1, pt2]):
                cv2.line(img_bgr, pt1, pt2, (0, 255, 255), 2)

        # Kulcspontok rajzolása
        for i, (x, y) in enumerate(keypoints):
            if 0 <= x < img_bgr.shape[1] and 0 <= y < img_bgr.shape[0]:
                cv2.circle(img_bgr, (int(x), int(y)), 4, (0, 0, 255), -1)

        cv2.imwrite(output_path, img_bgr)

    def _save_coordinates(self, file_obj, image_path, keypoints):
        """Koordináták mentése fájlba"""
        filename = os.path.basename(image_path)
        coords_str = " ".join(f"({x:.2f},{y:.2f})" for x, y in keypoints)
        file_obj.write(f"{filename}: {coords_str}\n")

    def _find_image_files(self, input_dir):
        """Képfájlok keresése a megadott könyvtárban"""
        extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.tiff']
        image_files = []
        for ext in extensions:
            image_files.extend(glob.glob(os.path.join(input_dir, ext)))
            image_files.extend(glob.glob(os.path.join(input_dir, ext.upper())))
        return image_files

    # További segédmetódusok a HRNet-hez
    def _get_affine_transform(self, center, scale, rot, output_size, inv=0):
        """Affin transzformáció számítása"""
        scale_tmp = scale * 200.0
        src_w, dst_w = scale_tmp[0], output_size[0]
        src_h, dst_h = scale_tmp[1], output_size[1]

        src = np.zeros((3, 2), dtype=np.float32)
        dst = np.zeros((3, 2), dtype=np.float32)

        src[0, :] = center
        dst[0, :] = [dst_w * 0.5, dst_h * 0.5]

        src_dir = self._get_dir([0, -src_h * 0.5], math.radians(rot))
        dst_dir = np.array([0, -dst_h * 0.5], np.float32)

        src[1, :] = center + src_dir
        dst[1, :] = dst[0, :] + dst_dir

        src[2:, :] = self._get_3rd_point(src[0, :], src[1, :])
        dst[2:, :] = self._get_3rd_point(dst[0, :], dst[1, :])

        return cv2.getAffineTransform(np.float32(src if not inv else dst),
                                    np.float32(dst if not inv else src))

    def _get_final_preds(self, batch_heatmaps, center, scale):
        """Heatmap-ből végső pozíciók kinyerése"""
        coords, maxvals = self._get_max_preds(batch_heatmaps)

        if self.hrnet_cfg['TEST']['POST_PROCESS']:
            for n in range(coords.shape[0]):
                for p in range(coords.shape[1]):
                    hm = batch_heatmaps[n][p]
                    px, py = int(math.floor(coords[n][p][0] + 0.5)), int(math.floor(coords[n][p][1] + 0.5))
                    if 1 < px < self.hrnet_cfg['MODEL']['HEATMAP_SIZE'][0] - 1 and 1 < py < self.hrnet_cfg['MODEL']['HEATMAP_SIZE'][1] - 1:
                        diff = np.array([hm[py][px+1] - hm[py][px-1], hm[py+1][px] - hm[py-1][px]])
                        coords[n][p] += np.sign(diff) * .25

        preds = coords.copy()
        for i in range(coords.shape[0]):
            preds[i] = self._transform_preds(coords[i], center[i], scale[i],
                                           self.hrnet_cfg['MODEL']['HEATMAP_SIZE'])
        return preds, maxvals

    def _get_max_preds(self, batch_heatmaps):
        """Maximális értékek és pozíciók kinyerése"""
        batch_size, num_joints, _, width = batch_heatmaps.shape
        heatmaps_reshaped = batch_heatmaps.reshape((batch_size, num_joints, -1))
        idx = np.argmax(heatmaps_reshaped, 2)
        maxvals = np.amax(heatmaps_reshaped, 2)

        preds = np.tile(idx.reshape((batch_size, num_joints, 1)), (1, 1, 2)).astype(np.float32)
        preds[:,:,0] = (preds[:,:,0]) % width
        preds[:,:,1] = np.floor((preds[:,:,1]) / width)

        pred_mask = np.tile(np.greater(maxvals, 0.0), (1, 1, 2)).astype(np.float32)
        return preds * pred_mask, maxvals.reshape((batch_size, num_joints, 1))

    def _transform_preds(self, coords, center, scale, output_size):
        """Koordináták transzformálása az eredeti kép méretére"""
        target_coords = np.zeros(coords.shape)
        trans = self._get_affine_transform(center, scale, 0, output_size, inv=1)

        for p in range(coords.shape[0]):
            target_coords[p, 0:2] = self._affine_transform(coords[p, 0:2], trans)
        return target_coords

    def _affine_transform(self, pt, t):
        """Pont affin transzformációja"""
        new_pt = np.array([pt[0], pt[1], 1.], dtype=np.float32).T
        return np.dot(t, new_pt)[:2]

    def _get_dir(self, src_point, rot_rad):
        """Irányvektor kiszámítása rotációval"""
        sn, cs = np.sin(rot_rad), np.cos(rot_rad)
        return [src_point[0] * cs - src_point[1] * sn, src_point[0] * sn + src_point[1] * cs]

    def _get_3rd_point(self, a, b):
        """Harmadik pont kiszámítása két pontból"""
        direct = a - b
        return b + np.array([-direct[1], direct[0]], dtype=np.float32)


In [7]:
# Példa a fent létrehozott saját osztály használatára:
if __name__ == "__main__":

    # Detektor létrehozása
    detector = InfantPoseDetector_AGMA()

    # Egyetlen kép feldolgozása
    # keypoints = detector.process_image("test_image.jpg", output_dir="output")

    # Teljes könyvtár feldolgozása
    detector.process_batch(
        input_dir="images/train",
        output_dir="images/out_train",
        coords_file="result_coords.txt"
    )

Feldolgozás (1/1106): 00GG_nicowlcam06-2024-10-27-11-50-24_frame.jpg
Hiba a kép feldolgozása során: operands could not be broadcast together with shapes (1,17,2) (1,1,34) 
Feldolgozás (2/1106): 00kk-nicowlcam09-2025-02-25-23-53-12_frame.jpg
Feldolgozás (3/1106): 00kk-nicowlcam09-2025-02-25-17-23-13_frame.jpg
Feldolgozás (4/1106): 00kk-nicowlcam09-2025-02-25-10-05-09_frame.jpg
Feldolgozás (5/1106): 00kk-nicowlcam09-2025-02-25-04-15-09_frame.jpg
Feldolgozás (6/1106): 00GG_nicowlcam06-2024-10-27-03-50-25_frame.jpg
Hiba a kép feldolgozása során: operands could not be broadcast together with shapes (1,17,2) (1,1,34) 
Feldolgozás (7/1106): 00kk-nicowlcam09-2025-02-25-01-05-09_frame.jpg
Feldolgozás (8/1106): 00kk-nicowlcam09-2025-02-25-12-55-09_frame.jpg
Feldolgozás (9/1106): 00GG_nicowlcam06-2024-10-25-19-20-25_frame.jpg
Hiba a kép feldolgozása során: operands could not be broadcast together with shapes (1,17,2) (1,1,34) 
Feldolgozás (10/1106): 00kk-nicowlcam09-2025-02-26-00-03-12_frame.jpg
